In [1]:
import nannyml as nml
import numpy as np

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
reference_df = nml.load_synthetic_binary_classification_dataset()[0]
analysis_df = nml.load_synthetic_binary_classification_dataset()[1]

estimator = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    metrics=["business_cost"],
    chunk_size=5000,
    problem_type='classification_binary',
    business_cost_matrix = np.array([[-5, 10], [5, 0]]),
)

estimator.fit(reference_df)

results = estimator.estimate(analysis_df)

CalculatorException: failed while fitting nannyml.performance_estimation.confidence_based.cbpe.CBPE.
unsupported operand type(s) for +: 'NoneType' and 'NoneType'